In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from sklearn.linear_model import LinearRegression


In [ ]:

# There are 26 columns: 1-2 are unit/cycle, 3-5 operational settings, 6-26 are sensors (1 to 21)
# The columns in the txt files are:
#   1: unit
#   2: cycle
#   3: op_setting_1
#   4: op_setting_2
#   5: op_setting_3
#   6-26: sensor_1 to sensor_21
col_names = [
    'unit', 'cycle',
    'op_setting_1', 'op_setting_2', 'op_setting_3',
    'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
    'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10',
    'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15',
    'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21'
]
train = pd.read_csv('../data/CMAPSSData/train_FD001.txt', sep='\\s+', header=None, names=col_names, index_col=None)
test = pd.read_csv('../data/CMAPSSData/test_FD001.txt', sep='\\s+', header=None, names=col_names)
rul = pd.read_csv('../data/CMAPSSData/RUL_FD001.txt', sep='\\s+', names=['RUL'])


In [ ]:
print(train.shape)
print(train.head())


In [ ]:
train.info()


In [ ]:
train.describe()

In [ ]:
train['sensor_6'].describe()

In [ ]:

# Check for missing values
print(train.isnull().sum())


In [ ]:
for col in train.columns:
    if col in ['op_setting_1', 'op_setting_2']:
        continue  # Skip op_setting_1 and op_setting_2 as these are important settings
    # Consider a column "constant" if it has only one unique value,
    # or if its standard deviation is very small (e.g., < 0.002), 
    # or if its min and max are extremely close (difference < 0.02)
    std = train[col].std()
    min_val = train[col].min()
    max_val = train[col].max()
    if (
        train[col].nunique() == 1
        or std < 0.002
        or abs(max_val - min_val) < 0.02
    ):
        print(f"{col} is constant or nearly constant and should be dropped.")


Sensors like sensor_1, sensor_5, op_setting_3, sensor_6, sensor_10, sensor_16, sensor_18, sensor_19 show either 0 std or a tiny range—these are likely constant and not useful as features.

Drop those for modeling, as they don't provide predictive signal.

In [ ]:
drop_cols = [
    'sensor_1', 'sensor_5', 'sensor_6', # Added sensor 6
    'sensor_10', 'sensor_16', 
    'sensor_18', 'sensor_19', 'op_setting_3'  # op_setting_3 appears to be constant at 100.0
    # 'op_setting_2' # Removed from the list as it has a big range of values though standard deviation is around 0
]
train = train.drop(columns=drop_cols)

In [ ]:
# Number of unique engines
print("Train units:", train['unit'].nunique())
print("Test units:", test['unit'].nunique())

# Distribution of cycles per engine
cycles_per_engine = train.groupby('unit')['cycle'].max()
print(cycles_per_engine.describe())

cycles_per_engine.hist(bins=30)
plt.xlabel('Number of cycles until failure')
plt.ylabel('Number of Engines')
plt.title('Engine Lifetime Distribution (FD001)')
plt.show()


For each row in the training set, generate the RUL label as $$\mathrm{RUL} = \max\_cycle_{\mathrm{engine}} - \mathrm{cycle}$$


In [ ]:
train['max_cycle'] = train.groupby('unit')['cycle'].transform('max')
train['RUL'] = train['max_cycle'] - train['cycle']
processed = train.drop(columns='max_cycle')

processed

Rolling Feature Engineering Functions

In [ ]:

def add_rolling_stats(df, group_col, feature_cols, window=5):
    """
    Adds rolling mean, std, min, max, and trend for each feature in feature_cols grouped by group_col.
    """
    result_df = df.copy()
    for feature in feature_cols:
        # Rolling mean
        result_df[f'{feature}_mean{window}'] = (
            result_df.groupby(group_col)[feature]
            .rolling(window=window, min_periods=1)
            .mean().reset_index(0, drop=True)
        )
        # Rolling std
        result_df[f'{feature}_std{window}'] = (
            result_df.groupby(group_col)[feature]
            .rolling(window=window, min_periods=1)
            .std().reset_index(0, drop=True)
        )
        # Rolling min
        result_df[f'{feature}_min{window}'] = (
            result_df.groupby(group_col)[feature]
            .rolling(window=window, min_periods=1)
            .min().reset_index(0, drop=True)
        )
        # Rolling max
        result_df[f'{feature}_max{window}'] = (
            result_df.groupby(group_col)[feature]
            .rolling(window=window, min_periods=1)
            .max().reset_index(0, drop=True)
        )
        # Rolling trend/slope
        def rolling_trend(x):
            idx = np.arange(len(x)).reshape(-1,1)
            if len(x) < 2:
                return 0.0
            return LinearRegression().fit(idx, x.values.reshape(-1,1)).coef_[0]
        result_df[f'{feature}_trend{window}'] = (
            result_df.groupby(group_col)[feature]
            .rolling(window=window, min_periods=2)
            .apply(rolling_trend, raw=False)
            .reset_index(0, drop=True)
        )
    return result_df


Sensor Selection and Correlation Functions

In [ ]:
def plot_sensor_vs_cycle(df, unit_col, sensors, unit_id):
    """
    Plots sensor values over cycles for a given engine/unit.
    """
    for sensor in sensors:
        plt.figure(figsize=(8, 3))
        subset = df[df[unit_col] == unit_id]
        plt.plot(subset['cycle'], subset[sensor])
        plt.title(f'{sensor} over cycles - Engine {unit_id}')
        plt.xlabel('Cycle')
        plt.ylabel(sensor)
        plt.show()

def plot_sensor_vs_rul(df, unit_col, sensors, unit_id):
    """
    Plots sensor values against RUL for a given engine/unit.
    """
    for sensor in sensors:
        plt.figure(figsize=(8, 3))
        subset = df[df[unit_col] == unit_id]
        plt.plot(subset['RUL'], subset[sensor])

        plt.title(f'{sensor} vs. RUL - Engine {unit_id}')
        plt.xlabel('RUL')
        plt.ylabel(sensor)
        plt.gca().invert_xaxis()
        plt.show()

def sensor_rul_correlation(df, sensor_cols):
    """
    Computes correlation between each sensor and RUL, prints sorted (desc).
    """
    correlations = []
    for sensor in sensor_cols:
        corr = df[sensor].corr(df['RUL'])
        correlations.append((sensor, corr))
    correlations.sort(key=lambda x: abs(x[1]), reverse=True)
    print("Sensor correlations with RUL (descending):")
    for sensor, corr in correlations:
        print(f"{sensor}: {corr:.3f}")
    return correlations  # for downstream filtering


In [ ]:
# sensors_to_engineer filtered for your dataset (excluding dropped/constant ones)
sensors_to_engineer = [col for col in processed.columns if col.startswith('sensor') and col not in drop_cols]

# Add rolling statistics (window = 5)
processed_eng = add_rolling_stats(processed, group_col='unit', feature_cols=sensors_to_engineer, window=5)

# Visualizations for unit 1 (or any engine of interest)
plot_sensor_vs_cycle(processed_eng, unit_col='unit', sensors=sensors_to_engineer, unit_id=30)
plot_sensor_vs_rul(processed_eng, unit_col='unit', sensors=sensors_to_engineer, unit_id=30)

# Correlations to help select sensors
correlations = sensor_rul_correlation(processed_eng, sensors_to_engineer)


#### Interpretation of RUL Correlation Results
Sensors most correlated with RUL:
(Magnitude reflects how strongly sensor readings change as an engine approaches failure.)

| Rank | Sensor     | Correlation with RUL | Importance        |
|------|------------|----------------------|-------------------|
| 1    | sensor_11  | -0.696               | Very Strong       |
| 2    | sensor_4   | -0.679               | Very Strong       |
| 3    | sensor_12  | 0.672                | Very Strong       |
| 4    | sensor_7   | 0.657                | Strong            |
| 5    | sensor_15  | -0.643               | Strong            |
| 6    | sensor_21  | 0.636                | Strong            |
| 7    | sensor_20  | 0.629                | Strong            |
| 8    | sensor_2   | -0.606               | Strong            |
| 9    | sensor_17  | -0.606               | Strong            |
| 10   | sensor_3   | -0.585               | Strong            |
| 11   | sensor_8   | -0.564               | Moderate-Strong   |
| 12   | sensor_13  | -0.563               | Moderate-Strong   |
| 13   | sensor_9   | -0.390               | Moderate          |
| 14   | sensor_14  | -0.307               | Medium            |

Negative correlations (e.g., sensor_11, sensor_4, etc.):
As the engine approaches failure (RUL decreases), sensor value decreases.

Positive correlations (e.g., sensor_12, sensor_7, etc.):
As the engine approaches failure, sensor value increases.

I've plotted sensor_11 and sensor_4 against RUL for engine 30.
The correlation analysis said these features have strong negative correlation with RUL across the whole dataset, so:

Expectation: As RUL decreases (i.e., engine ages), sensor value should decrease.

What you see: In the graphs, as RUL decreases (moving right to left), the sensor value actually increases.

#### Why This Apparent Contradiction?
1. Direction of the X-axis:
    In the graphs, RUL goes from 200 (left) to 0 (right).

    As time progresses (engine approaches failure), RUL decreases—so the rightmost end is near failure.
    As you move right, you get closer to failure.

2. Negative Correlation — What It Actually Means
    Negative correlation means:

    Across the entire dataset, when RUL is higher, the sensor value is lower.

    Or equivalently, as RUL decreases, the sensor value increases.

    This matches exactly what your plots are showing:
    For lower RUL (engine near failure, right side), sensor readings are higher than at high RUL.

#### How to Read These Plots:
* On your x-axis: left (high RUL) → right (low RUL)

* On your y-axis: sensor reading

As you go right (engine nears failure), the sensor value goes up.

This means:

* Lower RUL → Higher sensor reading
* Higher RUL → Lower sensor reading

This matches a negative correlation: when RUL is lower, the sensor value is higher (the two vary inversely).

#### Summary Table
| Correlation (RUL, Sensor) | What It Means                      | What To See in Plot                |
|---------------------------|------------------------------------|------------------------------------|
| Negative (e.g., -0.69)    | As RUL decreases, sensor ↑         | Right side of plot: sensor ↑       |
| Positive (e.g., +0.60)    | As RUL decreases, sensor ↓         | Right side of plot: sensor ↓       |

#### Conclusion
Negative correlation between RUL and the sensor value (e.g., corr = -0.69) means:

* As RUL decreases (engine gets closer to failure), sensor value increases.

* As RUL increases (engine is “healthier”), sensor value decreases.



In [ ]:
processed

In [ ]:
window = 5  # Try window = 10 for longer-term trend features
feature_sensors = ['sensor_11', 'sensor_7', 'sensor_12', 'sensor_15', 'sensor_21', 'sensor_4']  # as per RUL ranking

for sensor in feature_sensors:
    processed[f'{sensor}_rollmean'] = (
        processed.groupby('unit')[sensor]
        .rolling(window, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )
    processed[f'{sensor}_rollstd'] = (
        processed.groupby('unit')[sensor]
        .rolling(window, min_periods=1)
        .std()
        .reset_index(level=0, drop=True)
    )

def rolling_slope(series, window):
    def linreg(x):
        if len(x) < 2:
            return 0
        x_idx = np.arange(len(x))
        slope = np.polyfit(x_idx, x, 1)[0]
        return slope
    return series.rolling(window, min_periods=2).apply(linreg, raw=True)

for sensor in feature_sensors:
    processed[f'{sensor}_slope'] = (
        processed.groupby('unit')[sensor]
        .apply(lambda x: rolling_slope(x, window))
        .reset_index(level=0, drop=True)
    )


In [ ]:
sensor_cols = [col for col in processed.columns if col.startswith('sensor_')]
sensor_stds = processed[sensor_cols].std().sort_values(ascending=False)


plt.figure(figsize=(12,5))
sns.barplot(x=sensor_stds.index, y=sensor_stds.values)
plt.xticks(rotation=90)
plt.ylabel('Std Deviation')
plt.title('Sensor Standard Deviation (Variance)')
plt.show()


In [ ]:
active_sensors = [col for col in sensor_cols if processed[col].std() > 0.1]  # Adjust threshold as needed
corr = processed[active_sensors].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm', annot=False)
plt.title('Active Sensor Correlation Matrix')
plt.show()


In [ ]:
corr

* Sensor standard deviation plot:

    * Shows which sensors are active (high std), which are nearly constant (low std).

    * Drop sensors with very low variance—they add little or no predictive power.

* Correlation matrix:

    * Many sensors are strongly correlated (|corr| > 0.7–0.8).

    * Example: sensor_4, sensor_11, sensor_12, sensor_7, sensor_21 form a “core group” of highly correlated signals.

    * Redundant features inflate model complexity and can degrade performance (especially for linear models).

Robust, automated feature selection pipeline based on your criteria for sensor data, ready for production or iterative refinement.
This will:

* Drop low-variance features

* Select features with high absolute correlation to RUL

* Drop one of each highly correlated pair (by |corr| > 0.9), keeping the one with higher RUL-correlation

* Optionally, add trend/rolling features (but not when redundant)

In [ ]:

def automated_feature_selection(
    df, 
    target_col='RUL', 
    op_set_cols=['op_setting_1', 'op_setting_2'], 
    sensor_prefix='sensor_',
    std_threshold=0.1,
    top_k=8,
    corr_cutoff=0.9,
    add_rollmean=False,
    add_slope=False
):
    # 1. Find sensor columns
    sensor_cols = [col for col in df.columns if col.startswith(sensor_prefix) and df[col].dtype != 'object']

    # 2. Drop low-variance sensors
    active_sensors = [col for col in sensor_cols if df[col].std() > std_threshold]

    # 3. Correlation to RUL (absolute values)
    sensor_rul_corr = {col: abs(df[col].corr(df[target_col])) for col in active_sensors}
    sorted_sensors = sorted(sensor_rul_corr, key=lambda col: sensor_rul_corr[col], reverse=True)
    # Keep top K RUL-informative sensors
    selected = sorted_sensors[:top_k]

    # 4. Remove one of each highly correlated pair (|corr| > corr_cutoff)
    keep = []
    for col in selected:
        redundant = False
        for k in keep:
            pairwise_corr = abs(df[[col, k]].corr().iloc[0,1])
            if pairwise_corr > corr_cutoff:
                redundant = True
                break
        if not redundant:
            keep.append(col)
    selected = keep

    # 5. Optional: add rollmean/slope features for selected sensors (non-redundant only)
    extra = []
    if add_rollmean:
        for col in selected:
            roll_col = f"{col}_rollmean"
            if roll_col in df.columns:
                extra.append(roll_col)
    if add_slope:
        for col in selected:
            slope_col = f"{col}_slope"
            if slope_col in df.columns:
                extra.append(slope_col)

    # 6. Always include operational settings, combine final feature list
    final_features = op_set_cols + selected + extra

    print("Selected features:", final_features)
    return final_features


In [ ]:
final_features = automated_feature_selection(
    processed,                   # DataFrame with sensor, op setting, and RUL columns
    target_col='RUL',
    op_set_cols=['op_setting_1', 'op_setting_2'],
    std_threshold=0.1,
    top_k=10,
    corr_cutoff=0.9,             # Remove highly intercorrelated sensors
    add_rollmean=True,
    add_slope=True
)

#### Modelling

In [ ]:
processed= processed.dropna()
processed.isna().sum()

Rolling mean/slope with min_periods=1: Usually, only the very first rows for each engine/unit will have enough values to fully calculate a rolling window of size 5 (or your window).

Rolling slope (min_periods=2): The very first entry per group is guaranteed to be NaN, since slope requires at least 2 points. For rolling mean, if you set min_periods=1, it should not produce nulls unless you have accidental group issues or missing data upstream.

Result: For each engine, the first (window-1) rows for rolling mean, and first row for slope, will have nulls.

Removal:
* Safest and most common: Simply remove all rows containing any NaNs after rolling/stat features are created.

* In this dataset, since these are just the very first few cycles per engine (before any significant degradation or signal), you lose little predictive info.

In [ ]:
from sklearn.model_selection import train_test_split

X = processed[final_features]
y = processed['RUL']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# MODEL PIPELINE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

pipeline = Pipeline([
    ('scaler', StandardScaler()),              # Normalize features
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
rmse = mean_squared_error(y_val, y_pred)
print(f'MAE: {mae:.2f}, RMSE: {rmse:.2f}')


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# X, y assumed to be your final feature matrix and label.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Candidate models and their parameter grids:
model_dict = {
    "LinearRegression": (
        LinearRegression(),
        {}
    ),
    "Ridge": (
        Ridge(),
        {'model__alpha': [0.01, 0.1, 1.0, 10.0]}
    ),
    "Lasso": (
        Lasso(max_iter=10000),
        {'model__alpha': [0.01, 0.1, 1.0, 10.0]}
    ),
    "RandomForest": (
        RandomForestRegressor(random_state=42),
        {'model__n_estimators': [100, 200], 'model__max_depth': [None, 8, 16]}
    ),
    "GradientBoosting": (
        GradientBoostingRegressor(random_state=42),
        {'model__n_estimators': [100, 200], 'model__max_depth': [3, 5]}
    ),
}

# Optionally add XGBoost if installed
try:
    from xgboost import XGBRegressor
    model_dict["XGBoost"] = (
        XGBRegressor(random_state=42, verbosity=0),
        {'model__n_estimators': [100, 200], 'model__max_depth': [3, 5]}
    )
except ImportError:
    print("XGBoost is not installed, skipping.")

results = {}

for model_name, (regressor, param_grid) in model_dict.items():
    print(f"GridSearchCV for {model_name}...")
    pipeline = Pipeline([
        ('scaler', StandardScaler()),   # Always scale for fair comparison
        ('model', regressor)
    ])
    gs = GridSearchCV(
        pipeline,
        param_grid=param_grid,
        cv=3,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        error_score="raise"
    )
    gs.fit(X_train, y_train)
    best_est = gs.best_estimator_
    y_pred = best_est.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    results[model_name] = {
        'best_params': gs.best_params_,
        'MAE': mae,
        'RMSE': rmse,
        'model': best_est
    }
    print(f"{model_name}: MAE={mae:.2f}, RMSE={rmse:.2f}, Best Params: {gs.best_params_}")


In [ ]:

print("\n=== Summary of Results ===")
for model_name, res in results.items():
    print(f"{model_name}: MAE={res['MAE']:.2f}, RMSE={res['RMSE']:.2f}, Best Params: {res['best_params']}")


#### Model Comparison Plot (MAE and RMSE)

In [ ]:
import plotly.graph_objects as go

# Collect results for plotting
model_names = list(results.keys())
mae_values = [results[m]['MAE'] for m in model_names]
rmse_values = [results[m]['RMSE'] for m in model_names]

fig = go.Figure(data=[
    go.Bar(name='MAE', x=model_names, y=mae_values, marker_color='indianred'),
    go.Bar(name='RMSE', x=model_names, y=rmse_values, marker_color='royalblue')
])
fig.update_layout(
    title='Model Comparison: MAE and RMSE',
    barmode='group',
    yaxis_title='Error Value',
    template='plotly_white'
)
fig.show()


#### Residual Histogram for Each Model

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for model_name, res in results.items():
    best_model = res['model']
    preds = best_model.predict(X_val)
    residuals = y_val - preds
    import numpy as np
    from scipy.stats import gaussian_kde

    # Histogram trace
    # Define a color for each model for consistent coloring between histogram and KDE
    import plotly.colors as pc
    color_palette = pc.qualitative.Plotly
    color_idx = model_names.index(model_name) % len(color_palette)
    color = color_palette[color_idx]

    # Histogram trace (with model-specific color)
    fig.add_trace(
        go.Histogram(
            x=residuals,
            nbinsx=50,
            name=f"{model_name} Histogram",
            opacity=0.4,
            marker=dict(color=color, line=dict(width=0)),
            showlegend=True,
        )
    )

    # KDE curve trace (with same color)
    kde = gaussian_kde(residuals)
    x_range = np.linspace(residuals.min(), residuals.max(), 200)
    # Scale KDE to match histogram area
    hist_area = len(residuals) * (residuals.max() - residuals.min()) / 50
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=kde(x_range) * hist_area,
            mode='lines',
            name=f"{model_name} KDE",
            line=dict(width=3, color=color),
            showlegend=True,
        )
    )

fig.update_layout(
    title="Residuals Histogram for Each Model",
    template='plotly_white',
    barmode='overlay',
    xaxis_title="Residual (True RUL - Predicted RUL)",
    yaxis_title="Count",
    legend_title="Model",
    height=500,
    width=800
)
fig.show()


#### Feature Importance Barplots (Tree-Based Models Only)

In [ ]:
# Combine all feature importances into a single interactive plot with legend

import plotly.graph_objects as go

# Collect feature importances for all tree-based models
feature_importance_traces = []
for model_name, res in results.items():
    model = res['model']
    if hasattr(model.named_steps['model'], 'feature_importances_'):
        importances = model.named_steps['model'].feature_importances_
        trace = go.Bar(
            x=importances,
            y=X_val.columns,
            orientation='h',
            name=model_name,
            visible=True if len(feature_importance_traces) == 0 else 'legendonly'  # Show first by default
        )
        feature_importance_traces.append(trace)

if feature_importance_traces:
    fig = go.Figure(feature_importance_traces)
    fig.update_layout(
        title="Feature Importance Comparison (Tree-Based Models)",
        barmode='group',
        template='plotly_white',
        yaxis={'categoryorder': 'total ascending'},
        xaxis_title='Importance',
        yaxis_title='Feature',
        legend_title='Model'
    )
    fig.show()
else:
    print("No tree-based models with feature_importances_ found.")


#### Hyper-parameter Tunning

In [ ]:
from sklearn.model_selection import GridSearchCV

# Use all available data for final training
X_final = processed[final_features]
y_final = processed['RUL']

param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [6, 8, 12],
    'model__max_features': ['sqrt', 'log2'],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(random_state=42))
])
grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1, error_score='raise')
grid.fit(X_final, y_final)
print("Best params:", grid.best_params_)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import joblib

# Update best parameters for your RandomForest
best_params = {
    'n_estimators': 200,
    'max_depth': 8,
    'max_features': 'sqrt',
    'min_samples_split': 10,
    'min_samples_leaf': 4,
    'random_state': 42
}

# Use your chosen features and RUL column
X_final = processed[final_features]
y_final = processed['RUL']

# Full model pipeline
final_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(**best_params))
])

# Train on all data
final_pipeline.fit(X_final, y_final)

# Save model pipeline for deployment/inference
joblib.dump(final_pipeline, "../model/model_rf_pipeline_best.joblib")
print("Final model retrained and saved as model_rf_pipeline_best.joblib")


### Testing

In [ ]:
import pandas as pd
import numpy as np
import joblib

# ---- PARAMETERS ----
WINDOW = 5

final_features = [
    'op_setting_1', 'op_setting_2',
    'sensor_4_rollmean', 'sensor_12', 'sensor_7', 'sensor_21', 'sensor_20',
    'sensor_12_rollmean', 'sensor_7_rollmean', 'sensor_21_rollmean',
    'sensor_12_slope', 'sensor_7_slope', 'sensor_21_slope'
]

# --- UTILS ---
def add_rolling_mean(df, feature, group='unit', window=5):
    return (
        df.groupby(group)[feature]
        .rolling(window, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )

def rolling_slope(series, window):
    def linreg(x):
        if len(x) < 2:
            return np.nan
        x_idx = np.arange(len(x))
        slope = np.polyfit(x_idx, x, 1)[0]
        return slope
    return series.rolling(window, min_periods=2).apply(linreg, raw=True)

def add_rolling_slope(df, feature, group='unit', window=5):
    return (
        df.groupby(group)[feature]
        .apply(lambda x: rolling_slope(x, window))
        .reset_index(level=0, drop=True)
    )

# --- 1. LOAD DATA ---
test_raw = pd.read_csv('../data/CMAPSSData/test_FD001.txt', sep='\\s+', header=None, names=col_names)
col_names = ['unit', 'cycle', 'op_setting_1', 'op_setting_2', 'op_setting_3'] + [f'sensor_{i}' for i in range(1, 22)]
test_raw.columns = col_names

# --- 2. DROP UNUSED CONSTANT COLUMNS (must match your training pipeline!) ---
drop_cols = ['op_setting_3', 'sensor_1', 'sensor_5', 'sensor_6', 'sensor_10', 'sensor_16', 'sensor_18', 'sensor_19']
test = test_raw.drop(columns=drop_cols)

# --- 3. COMPUTE ROLLING MEANS ---
for sensor in ['sensor_4', 'sensor_12', 'sensor_7', 'sensor_21']:
    test[f'{sensor}_rollmean'] = add_rolling_mean(test, sensor, window=WINDOW)

# --- 4. COMPUTE ROLLING SLOPE ---
for sensor in ['sensor_12', 'sensor_7', 'sensor_21']:
    test[f'{sensor}_slope'] = add_rolling_slope(test, sensor, window=WINDOW)

# --- 5. DROP ROWS WITH ANY NULL IN THE SELECTED FEATURES ---
test = test.dropna(subset=final_features)

# --- 6. FOR EACH ENGINE, RETAIN ONLY THE LAST CYCLE (after feature engineering) ---
test_last = test.groupby('unit').tail(1).reset_index(drop=True)

# --- 7. ALIGN FINAL FEATURES, PREDICT RUL ---
X_test = test_last[final_features]
model = joblib.load('../model/model_rf_pipeline_best.joblib')  # Or your specific filename

y_pred = model.predict(X_test)

# --- 8. LOAD TRUE RUL VALUES and COMPARE ---
true_rul = pd.read_csv('../data/CMAPSSData/RUL_FD001.txt', sep='\\s+', header=None, names=['true_RUL'])
y_true = true_rul['true_RUL'].values

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"Test set MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# For a quick comparison table
results_df = pd.DataFrame({
    'unit': test_last['unit'], 'true_RUL': y_true, 'predicted_RUL': y_pred
})
print(results_df.head(10))


In [ ]:
import plotly.express as px
import pandas as pd

# Assuming results_df with columns: unit, true_RUL, predicted_RUL
results_df['residual'] = results_df['true_RUL'] - results_df['predicted_RUL']
results_df['abs_error'] = results_df['residual'].abs()

# (a) Scatter True vs. Predicted RUL
fig = px.scatter(
    results_df, x='true_RUL', y='predicted_RUL', hover_data=['unit', 'residual'],
    title="Predicted RUL vs. True RUL (Test Set)",
    trendline="ols", template="plotly_white"
)
fig.add_shape(type='line', x0=0, y0=0, x1=results_df['true_RUL'].max(), y1=results_df['true_RUL'].max(),
              line=dict(color='gray', dash='dash'))
fig.update_layout(
    xaxis_title='True RUL', yaxis_title='Predicted RUL',
    showlegend=False
)
fig.show()

# (b) Residual Histogram
fig2 = px.histogram(
    results_df, x='residual', nbins=30,
    title="Distribution of Residuals (True RUL - Predicted RUL)",
    labels={'residual':'Residual (Error)'},
    marginal="box", template="plotly_white"
)
fig2.update_layout(bargap=0.2)
fig2.show()

# (c) Absolute Error Histogram
fig3 = px.histogram(
    results_df, x='abs_error', nbins=30,
    title="Distribution of Absolute Errors |True RUL - Predicted RUL|",
    labels={'abs_error':'Absolute Error'},
    marginal="box", template="plotly_white"
)
fig3.update_layout(bargap=0.2)
fig3.show()


#### Interpretation of Model Performance on Test Set
1. **Predicted RUL vs. True RUL Scatter Plot**
* What it shows:
    Each point represents a test engine at its final recorded cycle, with its true Remaining Useful Life (RUL) on the x-axis and your model’s predicted RUL on the y-axis.

    * Dashed gray line:
        Perfect predictions (Predicted RUL = True RUL) would fall exactly on this line.

    * Blue trend line:
        Shows the linear regression fit of your model’s predictions versus the ideal.

    **Interpretation:**
    
    Most points are reasonably close to the diagonal, indicating good model performance. However, some points, especially for high RUL values, are predicted as larger or smaller than true RUL, showing some over-/under-estimation on certain engines, which is common in this complex prediction task.

2. **Residual Distribution Plot**
* What it shows:
    Histogram of residuals (True RUL – Predicted RUL), with a notched box plot above.

    **Interpretation:**

    * The center of the distribution is near zero, which is ideal and indicates no systematic bias.

    * The spread is moderately symmetric but shows some negative skew (more underestimation; i.e., the model sometimes predicts lower RUL than actual).

    * Most residuals are within ±40 cycles.

    * A few residuals are more extreme—these are outliers where the model under/over-predicted more strongly, which often occurs for irregular or highly variable test cases.

3. **Absolute Error Distribution Plot**
* What it shows:
    Histogram and box plot of absolute errors |True RUL – Predicted RUL|.

    **Interpretation:**

    * Most absolute errors are below 40 cycles, with a concentration in the 10–30 range.

    * The box plot confirms that the median absolute error is low and most predictions are within a reasonable error bound.

    * The presence of some larger errors (outliers) indicates that, while the model handles the majority of cases well, a few engines are more difficult to predict exactly—likely due to unique sensor progression or boundary RUL values.




#### Summary of Model Evaluation:

The model demonstrates strong predictive performance on the test set, as evidenced by the error metrics (Test MAE: 25.31, RMSE: 33.44) and the distributional plots. The scatter plot of predicted vs. true RUL values shows that most predictions are close to the ideal line, confirming the model's ability to track the actual degradation pattern of most engines.

The residual and absolute error histograms indicate that the majority of prediction errors lie within a reasonable range, and there is no obvious systematic over- or under-prediction. While a few predictions result in larger errors, the central tendency and spread are both appropriate for the complexity of the task and the information available.

Conclusion:
With the inclusion of rolling statistical features, the model substantially improves over baselines and predicts RUL with low mean errors. This demonstrates both the potential of data-driven prognostics and the effectiveness of carefully engineered features in supporting remaining useful life estimation for complex systems like aircraft engines.